# Simple, Phase 1, approximate flight profile simulation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import control

# Purdue AA451 library
from scripts.jsbsim_utils import Logger, trim, simulate, linearize, rootlocus, clean_tf

In [2]:
aircraft_model = 'WaveRipper'

# Ground Trim

In [3]:
op_ground, props = trim(
    aircraft=aircraft_model,
    ic={
        'ic/vt-fps': 0,
        'ic/h-agl-ft': 5,
        'ic/psi-true-deg': 280,
        'fcs/left-brake-cmd-norm': 1,
        'fcs/right-brake-cmd-norm': 1,
        'fcs/center-brake-cmd-norm': 1,
    },
    design_vector=['ic/theta-rad', 'ic/h-agl-ft'],
    x0=[0, 0],
    verbose=True,
    method='Nelder-Mead', # works better with ground interaction
    ftol=1e-1, # TODO: once full aircraft params are specified, we can have finer fun tolerence
)
op_ground



     JSBSim Flight Dynamics Model v1.1.8 [GitHub build 588/commit c943f83deeb3e14bed7939ac65dfac789a7a0181] Jul 30 2021 15:04:27
            [JSBSim-ML v2.0]

JSBSim startup beginning ...

 final_simplex: (array([[-0.00398416, -0.0198497 ],
       [-0.00397723, -0.01982051],
       [-0.00397813, -0.01978912]]), array([0.0010305 , 0.00103479, 0.00109463]))
           fun: 0.0010305007323130465
       message: 'Optimization terminated successfully.'
          nfev: 78
           nit: 41
        status: 0
       success: True
             x: array([-0.00398416, -0.0198497 ])



  The aerodynamic moment axis system has been set by default to the bodyXYZ system.


{'ic/vt-fps': 0,
 'ic/h-agl-ft': -0.01984969996708967,
 'ic/psi-true-deg': 280,
 'fcs/left-brake-cmd-norm': 1,
 'fcs/right-brake-cmd-norm': 1,
 'fcs/center-brake-cmd-norm': 1,
 'ic/theta-rad': -0.0039841619774816615}

In [4]:
op_ground

{'ic/vt-fps': 0,
 'ic/h-agl-ft': -0.01984969996708967,
 'ic/psi-true-deg': 280,
 'fcs/left-brake-cmd-norm': 1,
 'fcs/right-brake-cmd-norm': 1,
 'fcs/center-brake-cmd-norm': 1,
 'ic/theta-rad': -0.0039841619774816615}

In [5]:
log_ground = simulate(
    aircraft=aircraft_model,
    op_0=op_ground,
    tf=5,
    realtime=False, verbose=True)


  The aerodynamic moment axis system has been set by default to the bodyXYZ system.


## Cruise Trim

In [6]:
op_cruise, props = trim(
    aircraft=aircraft_model,
    # For a list of initial conditions available:
    # https://jsbsim-team.github.io/jsbsim/classJSBSim_1_1FGInitialCondition.html
    ic={
        'ic/gamma-deg': 0,                          # Flightpath angle initial condition in degrees
        'ic/vt-fps': 600,                           # True airspeed initial condition in feet/second
        'ic/h-sl-ft': 10000,                        # Height above sea level initial condition in feet
        # 'gear/gear-cmd-norm': 0,
    },
    design_vector=[
        'fcs/throttle-cmd-norm',
        'fcs/elevator-cmd-norm',
        'fcs/rudder-cmd-norm',
        'fcs/aileron-cmd-norm',
        'ic/alpha-rad',
        'ic/beta-rad',
    ],
    # method='SLSQP',
    method='Nelder-Mead',
    # eq_constraints= [
    #     lambda fdm: fdm['accelerations/udot-ft_sec2'],
    #     lambda fdm: fdm['accelerations/vdot-ft_sec2'],
    #     # lambda fdm: fdm['accelerations/wdot-ft_sec2'],
    #     # lambda fdm: fdm['accelerations/pdot-rad_sec2'],
    #     lambda fdm: fdm['accelerations/qdot-rad_sec2'],
    #     lambda fdm: fdm['accelerations/rdot-rad_sec2'],
    # ],
    cost=lambda fdm: fdm['fcs/throttle-cmd-norm'],
    x0=[0.5, 0, 0, 0, 0, 0],
    verbose=False,
    bounds=[[0, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1], [-1, 1]],
    tol=1e-3
)
op_cruise


  The aerodynamic moment axis system has been set by default to the bodyXYZ system.


{'ic/gamma-deg': 0,
 'ic/vt-fps': 600,
 'ic/h-sl-ft': 10000,
 'fcs/throttle-cmd-norm': 0.0,
 'fcs/elevator-cmd-norm': 7.79320987654322e-05,
 'fcs/rudder-cmd-norm': 0.0008557098765432094,
 'fcs/aileron-cmd-norm': 0.0013742283950617285,
 'ic/alpha-rad': 0.0003804012345679012,
 'ic/beta-rad': 0.0005367798353909464}

In [7]:
log_cruise = simulate(
    aircraft=aircraft_model,
    op_0=op_cruise,
    tf=10,
    realtime=False)


  The aerodynamic moment axis system has been set by default to the bodyXYZ system.


In [8]:
op_cruise_auto = dict(op_cruise)
#op_cruise_auto['ic/theta-deg'] = 0
#op_cruise_auto['ic/phi-deg'] = 0
op_cruise_auto['ap/roll-enable'] = 1
op_cruise_auto['ap/pitch-enable'] = 1
op_cruise_auto['ap/yaw-enable'] = 1
op_cruise_auto['ap/h-enable'] = 1
op_cruise_auto['ap/heading-cmd-deg'] = 260
op_cruise_auto['ap/h-sl-cmd-ft'] = 10000

log_cruise_auto = simulate(
    aircraft='F-35B-2',
    op_0=op_cruise_auto,
    tf=40,
    realtime=False)